In [1]:
# Load common imports
%run ./CommonImports.ipynb

In [2]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def create_original_df(base_currency="EUR"):
    """Create the original dataframe based on ECB data.
   
    Parameters
    ----------
    base_currency : str
        The currency that is used to express other currency exchange rates."""
    c = CurrencyConverter("http://www.ecb.int/stats/eurofxref/eurofxref-hist.zip", 
                          fallback_on_wrong_date=True, 
                          fallback_on_missing_rate=True,
                          decimal=True)

    start_date = date(2000, 1, 1)
    end_date = date.today()

    date_list = []

    for single_date in daterange(start_date, end_date):

        cur_list = []
        cur_list.append(single_date)

        for currency in c.currencies:

            value = c.convert(1, base_currency, currency, date=single_date)
            cur_list.append(float(value))

        date_list.append(cur_list)

    columns_list = ["date"] + list(c.currencies)

    df = pd.DataFrame(date_list, columns=columns_list)
    df = df.sort_values(by="date", ascending=False)
    
    df["date"] = df["date"].map(lambda x: datetime(x.year, x.month, x.day, 0, 0))
    
    return df.sort_values("date").set_index("date")

def split_data(df, train_start, test_start, validation_start, validation_end):
    # Create dev (train and test) and validation set
    train = df.query("date >= @train_start and date < @test_start")
    test = df.query("date >= @test_start and date < @validation_start")
    dev = pd.concat([train, test])
    val = df.query("date >= @validation_start and date < @validation_end")
    return train, test, dev, val

In [3]:
# Create dictionairy with currencies per continent (Russia seen as European country)
currencies_per_continent = {"Africa": ["ZAR"],
                            "Asia": ["CNY","HKD","IDR","JPY","KRW","MYR","PHP","SGD","THB","TRY"],
                            "Europe": ["CHF","CZK","DKK","EUR","GBP","HRK","HUF","NOK","PLN","RON", "RUB","SEK"],
                            "North America": ["CAD","USD"],
                            "Oceania": ["AUD","NZD"],
                            "South America": []}

# Define currencies of interest
p13_currencies = ['USD','EUR','GBP',
                  'CHF','CNY','JPY',
                  'CAD','SEK','NOK',
                  'SGD','AUD','NZD',
                  'ZAR']

# Define list of (free-)floating currencies
# Source: https://en.wikipedia.org/wiki/List_of_countries_by_exchange_rate_regime#US_dollar_as_exchange_rate_anchor 
ff_currencies = ['AUD','CAD','JPY',
                 'NOK','PLN','SEK',
                 'GBP','EUR','RUB',
                 'USD']
f_currencies = ['HUF','IDR','KRW',
                'NZD','PHP','RON',
                'ZAR','THB','TRY']

# Create dataframe with pre-defined base currency
df = create_original_df("EUR")
obsolete_currencies = ["ISK", "MXN", "ROL", "BRL", "CYP", "EEK", "INR", "ILS", "SIT", "BGN", "LTL", "MTL", "TRL", "LVL", "SKK"]
all_currencies = [c for c in df.columns if c not in obsolete_currencies]

In [6]:
# Trading volumes
# Source: https://en.wikipedia.org/wiki/Foreign_exchange_market (added HRK as 0.1)
trading_volumes = {"USD": 88.3,
                   "EUR": 32.3,
                   "JPY": 16.8,
                   "GBP": 12.8,
                   "AUD": 6.8,
                   "CAD": 5.0,
                   "CHF": 5.0,
                   "CNY": 4.3,
                   "HKD": 3.5,
                   "NZD": 2.1,
                   "SEK": 2.0,
                   "KRW": 2.0,
                   "SGD": 1.8,
                   "NOK": 1.8,
                   "MXN": 1.7,
                   "INR": 1.7,
                   "RUB": 1.1,
                   "ZAR": 1.1,
                   "TRY": 1.1,
                   "BRL": 1.1,
                   "TWD": 0.9,
                   "DKK": 0.6,
                   "PLN": 0.6,
                   "THB": 0.5,
                   "IDR": 0.4,
                   "HUF": 0.4,
                   "CZK": 0.4,
                   "ILS": 0.3,
                   "CLP": 0.3,
                   "PHP": 0.3,
                   "AED": 0.2,
                   "COP": 0.2,
                   "SAR": 0.2,
                   "MYR": 0.1,
                   "RON": 0.1,
                   "HRK": 0.1,
                   "other": 2.1}

trading_volumes = {currency: (value / sum(trading_volumes.values())) for currency, value in trading_volumes.items()}

In [5]:
def get_libra_mix(currencies):
    result = pd.Series(np.zeros(len(currencies)), index=currencies)
    
    result["USD"] = .5
    result["EUR"] = .18
    result["JPY"] = .14
    result["GBP"] = .11
    result["SGD"] = .07
    
    return result

def get_sdr_mix(currencies):
    result = pd.Series(np.zeros(len(currencies)), index=currencies)
    
    result["USD"] = .4173
    result["EUR"] = .3093
    result["CNY"] = .1092
    result["JPY"] = .0833
    result["GBP"] = .0809
    
    return result